In [12]:
# Add project root to Python path
import os
import sys

# Get the absolute path to the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to Python path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Verify the path is added correctly
print(f"Added project root to Python path: {project_root}")

Added project root to Python path: /Users/brylie/code/remote-sensing-analysis


In [ ]:
import geemap
import ee
from config.areas import get_area
from src.extractors.sentinel import get_sentinel_data
from src.metrics.vegetation import calculate_evi, calculate_lai
from src.processors.preprocessing import add_date

Original source: https://github.com/krishnakafle/Blog_kaflekrishna/blob/main/GEE/LAI/LAI_calculations.ipynb

In [14]:
ee.Initialize()

In [ ]:
# Using get_area function from config.areas module to get Finland geometry
area_of_interest = get_area("finland")

In [ ]:
START_DATE = ee.Date("2024-10-01")
END_DATE = ee.Date("2024-11-01")

# Using get_sentinel_data function from src.extractors.sentinel module
sentinel_data = get_sentinel_data(START_DATE, END_DATE, area_of_interest)

# Apply EVI calculation to the fetched data
sentinel_data_with_evi = sentinel_data.map(calculate_evi)

In [ ]:
# Apply LAI calculation and create final composite image
lai_image = (
    sentinel_data_with_evi
    .map(calculate_lai)
    .map(add_date)
    .median()
    .clip(area_of_interest)
)

In [ ]:
palette = [
    "FFFFFF", "CE7E45", "DF923D", "F1B555", "FCD163", "99B718",
    "74A901", "66A000", "529400", "3E8601", "207401", "056201",
    "004C00", "023B01", "012E01", "011D01", "011301"
]

lai_palette = {"min": 0.5, "max": 3.5, "palette": palette}
map = geemap.Map()
map.centerObject(area_of_interest, 8)
map.addLayer(lai_image.select("LAI"), lai_palette, "LAI")

map.addLayerControl()
map

Map(center=[64.23487926926077, 26.152236411247713], controls=(WidgetControl(options=['position', 'transparent_…